# Create the inference wrappers

The notebook explains the role of Pre- and Postprocessing steps in the Pipeline in the aspect of GPURuntimeComponent.  
The final Python code is already created and provided in files [preprocessing.py](../src/preprocessing/preprocessing.py) and [postprocessing.py](../src/postprocessing/postprocessing.py), so the Pipeline can be created without executing this notebook.  
This case you can continue with building the Pipeline in notebook [30-CreatePipeline.ipynb](30-CreatePipeline.ipynb).

### Creating Preprocessing Step

The responsibility of the `Preprocessing` PythonComponent is to receive `ImageSet` payload and transform it to the input required by the `GPURuntimeComponent`.  
To do so, we need to be familiar with the `ImageSet` payload format. First, we need a picture to create this type of payload.

In [ ]:
from matplotlib import pyplot as plt
import numpy
from pathlib import Path

image_dir = Path('../data/processed')

images = list(str(f) for f in image_dir.rglob("*.jpg"))
images_count = len(images)

In [ ]:
import cv2

image = cv2.imread(images[0])  # BGR image 224x224x3
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB image 224x224x3

plt.imshow(image)

In [ ]:
image.shape

### Create ImageSet payload format

The method defined below creates the required format, and will be put into the payload with name `vision_payload` which will be provided by the `AI Inference Server` when an image is arrived from the selected camera through `Vision Connector Application`.

In [ ]:
def image_to_bayer(image_path):
    im = cv2.imread(image_path)  # RGB image 224x224x3
    (height, width) = im.shape[:2]
    (R,G,B) = cv2.split(im)

    bayerrg8 = numpy.zeros((height, width), numpy.uint8)

    # strided slicing for this pattern:
    #   R G
    #   G R
    bayerrg8[0::2, 0::2] = R[0::2, 1::2] # top left
    bayerrg8[0::2, 1::2] = G[0::2, 0::2] # top right
    bayerrg8[1::2, 0::2] = G[1::2, 1::2] # bottom left
    bayerrg8[1::2, 1::2] = B[1::2, 0::2] # bottom right

    return bayerrg8, width, height

In [ ]:
import datetime
import json

def create_imageset_dict(image_path):
    timestamp = datetime.datetime.now()
    
    image_bytes, width, height = image_to_bayer(image_path)

    return {
        "version": "1",  # version of the Metadata format
        "count": 1,  # Number of images on message
        "timestamp": timestamp.isoformat(),  # Camera acquisition time
        "detail": [{  # list of images with detailed information
            "id": str(image_path),  # unique image identifier. this case we use the filename of the original image
            "timestamp": str(timestamp.timestamp()),  # Timestamp provided by the camera
            "width": width,  # image width
            "height": height,  # image height
            "format": "BayerRG8",  # image format configure
            "metadata": "",  # optional extra information on image
            "image": image_bytes  # image binary with the given 'format'
        }]
    }

image_set_payload = {"vision_payload": create_imageset_dict(images[0])}
image_set_payload

### Extract image from ImageSet

Now the task of the PythonComponent `Preprocessing` is to extract image data from the payload, and create a flat array of float32 data.  
This time the original image is packaged into the payload in `BayerRG8` format, so the Preprocessing component converts it to a one-dimensional float32 array with using PIL Image and numpy transformations.

In [ ]:
WIDTH = 224
HEIGHT = 224

extracted = image_set_payload['vision_payload']
image_detail = extracted["detail"][0]

iuid = image_detail['id']
width = image_detail.get("width", WIDTH)
height = image_detail.get("height", HEIGHT)

image_data = numpy.frombuffer(image_detail['image'], dtype=numpy.uint8)  # BayerRG8, (width x height, )
print(image_data.shape)
image_data = image_data.reshape(width, height)                           # BayerRG8, (width, height)
print(image_data.shape)
image_data = cv2.cvtColor(image_data, cv2.COLOR_BayerRG2RGB)             # RGB, (width, height, 3)
print(image_data.shape)

image_array = image_data.astype(numpy.float32) / 255.  # normalizing into [0,1) range and converting to float32

print(f"image_array shape: {image_array.shape}")  # checking the image dimensions
plt.imshow(image_array)  # showing the image
plt.axis('off')

image_array = image_array.transpose(2,0,1)  # changing the shape from (224, 224, 3) to (3, 224, 224) as expected by the model
inputs = numpy.array(image_array.ravel())  # flattening the 3 dimensional array and adding to an empty batch
print(f"inputs shape {inputs.shape} and type '{inputs.dtype}'")  # checking the input shape and type

Once the tensor is created, we can create the response `output` from `input` (in the aspect of GPURuntimeComponent) and `iuid` to connect the images later in the `Postprocessing` step.

In [ ]:
output = {
    'iuid': iuid,
    'input': inputs
}

## Inferencing the model

To see the output of the model, let's inference the model to the created payload.  
First, we load the model into an InferenceSession and then executes the model against the created `inputs`, as it happens in `AI Inference Server`.  
More precisely, the `GPU Runtime` in `AI Inference Server` will reshape the one-dimensional `inputs` array, so we should do the same.

In [ ]:
from onnxruntime import InferenceSession
session = InferenceSession("../src/detection/1/model.onnx")

In [ ]:
boxes, labels, scores = session.run(["boxes","labels","scores"], {"input": inputs.reshape((1,3,224,224))})
boxes, labels, scores

## Create Postprocessing component

The responsibility of the `Postprocessing Python Component` is to receive the predictions from the `Detection GPU Runtime Component` and make a decision based on the results.  
In the previous step we created the output of the model, now let's transform it to the payload format as the AI Inference Server would pass it to the Postprocessing step.

In [ ]:
payload = {
    "iuid": str(images[0]),
    "boxes": boxes,
    "labels": labels,
    "scores": scores
}

Now we can form a judgment on the given predictions, now we are using a simple decision; if there is any scratches, or the number of the found holes is not 8, the board is wrong, other way it is ok.

In [ ]:
iuid = payload.get("iuid", None)
image_input = payload.get("input", None)
boxes = payload.get("boxes", None)
labels = payload.get("labels", None)
scores = payload.get("scores", None)

holes = 0
scratches = 0
for i in range(len(scores)):
    if scores[i] > 0.8:
        if labels[i] == 1:
            holes += 1
        if labels[i] == 2:
            scratches +=1

print(f"The board contains {holes} holes and {scratches} scratches.")
prediction = "OK" if holes == 8 and scratches == 0 else "DAMAGED"

response = {
    "prediction": prediction,
    "result": json.dumps({
            "prediction": prediction,
            "holes": holes,
            "scratces": scratches,
            "message": f"The board with id '{iuid}' contains {holes} holes and {scratches} scratches."
        })
}
response

Once the workflows are implemented and saved, you are ready to create the Pipeline in notebook [30-CreatePipeline.ipynb](./30-CreatePipeline.ipynb).  
The code snippets above are saved in Python scripts as
- [preprocessing.py](../src/preprocessing/preprocessing.py) and
- [postprocessing.py](../src/postprocessing/postprocessing.py)